In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [ ]:
list_url=[]; # 주소를 담을 빈 리스트 객체 생성

for i in range(1, 5001, 10):
    
    start=i # 변경해야할 파라미터 (starter)
    search_url="https://search.naver.com/search.naver?where=news&sm=tab_jum&query=%EC%9E%A5%EC%95%A0%EC%9D%B8%EB%8B%A8%EC%B2%B4+%EC%8B%9C%EC%9C%84&sort=1&start="+str(start)
    headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36'}
    res  = requests.get(search_url,headers=headers)
    cnt=1
    
    #reponse 객체에서 받은 텍스트 파싱 코드
    while True:
        try:
            l_url=res.text.split('</span><a href="')[cnt].split('" class')[0]  # 패턴을 찾아 텍스트 파싱
            list_url.append(l_url)
        
        except:
            break; # 주소가 더이상 없으면 브레이크
        cnt= cnt+1

In [ ]:
df = pd.DataFrame(list_url) # 리스트를 df로
df.to_csv('./네이버뉴스주소.csv') # 파일저장

In [2]:
df1=pd.read_csv('./네이버뉴스주소.csv',index_col=0) # 파일 다시 불러 오기
url_list = df1['0'].values.tolist() # df to list

In [3]:
df2=pd.DataFrame(columns=['url','title','context','comment_context','comment_nickname','comment_good','comment_bad'])

In [4]:
df2

,url,title,context,comment_context,comment_nickname,comment_good,comment_bad


In [5]:
rowsize=50
search_url="https://search.naver.com/search.naver?where=news&sm=tab_jum&query=%EC%9E%A5%EC%95%A0%EC%9D%B8%EB%8B%A8%EC%B2%B4+%EC%8B%9C%EC%9C%84&sort=1&start=5001"
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36',"referer": search_url}
timeout = 5

for i in range(0,len(url_list)):
    try:
        start=1
        page=1
        
        url=url_list[i]
        
        oid = url.split("oid=")[1].split("&")[0] #422
        aid = url.split("aid=")[1] #0000430957
        res_context  = requests.get(url,headers=headers) #response 객체 가져오기
        soup_context = BeautifulSoup(res_context.text, 'html.parser') # soup html파싱
        article_title=soup_context.find_all(id="articleTitle")[0].text # 기사 제목 
        article_context=soup_context.find_all(id="articleBodyContents")[0].text.replace('\n','').replace("\'","").replace("\w","").replace("&nbsp","").replace(";","").replace("\t","").replace("\r","").replace("\b","").replace("\f","").replace("\v","") #기사 내용
        tag= True
        while (tag):
            c_url="https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=news&templateId=view_society&pool=cbox5&_wr&_callback=jQuery112405573818535186856_1644652878711&lang=ko&country=KR&objectId=news"+oid+"%2C"+aid+"&categoryId=&pageSize="+str(rowsize)+"&indexSize=10&groupId=&listType=OBJECT&pageType=more&page="+str(page)+"&initialize=true&userType=&useAltSort=true&replyPageSize=20&sort=favorite&includeAllStatus=true&_=1644652878713"
            
            res  = requests.get(c_url,headers=headers)
            soup = BeautifulSoup(res.text, 'html.parser')
            
            comment_row =str(soup).split(',"totalRows":')[1].split(',"startIndex":')[0] # 댓글 갯수
        
            for row in range(start,rowsize):
                comment_context = str(soup).split('"contents":')[row].split('","userIdNo":')[0] #댓글 내용
                comment_context=re.sub(r"[^가-힣 ]","",comment_context)
                comment_id =str(soup).split('"userName":"')[row].split('","userProfileImage"')[0] #댓글 아이디
                comment_good =str(soup).split(',"sympathyCount":')[row].split(',"antipathyCount":')[0] #댓글 좋아요
                comment_bad =str(soup).split(',"antipathyCount":')[row].split(',"hideReplyButton":')[0] # 댓글 싫어요
                new_dic=dict( url = url, title = article_title, context= article_context, comment_context = comment_context,comment_nickname=comment_id,comment_good=comment_good,comment_bad=comment_bad)
                df2=df2.append(new_dic,ignore_index=True)
            page = page + 1
    except:
        pass;
    
df2.to_csv('./네이버_댓글.csv',encoding = 'utf-8') # 파일저장